In [ ]:
import time

def rollout(starting_cube, fsolver, verbose=False):
#{
    start = time.time()

    # These for algorithm metrics
    rot_time, h_time, i_time = [],[],[]

    cube = DirectCube(starting_cube)
    best_state = cube.state()
    all_moves = []
    best_moves = []
    best_value = 0

    rollout_cache={}
    for iteration in range(20):
    #{
        iteration_a = time.time()
        is_new_best_state = False

        # ROLLOUT (explores cyclic moves)
        for i, moves in enumerate(csolver.cycles):
        #{
            applied_moves = []
            rollout_cube = DirectCube(cube)
            #for j in range(len(moves)):
            for j in range(30):
            #{
                a = time.time()
                mv = moves[j%len(moves)]
                applied_moves.append(mv)
                rollout_cube.rotate(mv)
                rot_time.append(time.time() - a)

                b = time.time()
                k = tuple(rollout_cube.state())
                if k not in rollout_cache:
                    rollout_cache[k] = fsolver.order_heuristic(rollout_cube)                
                    if rollout_cache[k] > best_value:
                        best_value = rollout_cache[k]
                        best_moves = list(applied_moves)
                        best_state = rollout_cube.state()
                        is_new_best_state = True
                h_time.append(time.time() - b)
            #}
        #}
        
        # Store if new best cube rolled out
        if is_new_best_state:
            cube.reset(state=best_state)
            all_moves.extend(best_moves)
        
        i_time.append(time.time() - iteration_a)
        if not is_new_best_state: break
    #}
    
    if verbose: 
    #{
        total = time.time() - start
        print(f"Number of iterations: {len(i_time)}")
        print("Total run time:", total, "sec")
        print(f"Time rotating cube: {sum(rot_time)} sec (" + "{:.2f}".format(100*sum(rot_time)/total) + "%)")
        print(f"Time calcing heuristics: {sum(h_time)} sec (" + "{:.2f}".format(100*sum(h_time)/total) + "%)")
        print(f"Average time per iteration: {np.average(i_time)} sec")
        print(f"Rollout cache size: {len(rollout_cache)}")

        plt.figure(1)
        plt.plot(i_time)
        plt.title(f'Time Per Iteration')
        plt.ylabel('Seconds')
        plt.xlabel('Iteration')
    #}
    
    return cube, all_moves
#}

def generate_perturbations(cube, moves):
#{
    perturbations = []
    reference_cube = CfopCube()
    reverse_cube = CfopCube(cube)
    
    ishomelist = sum(reverse_cube.facelet_matrix[2:] == VectorCube._facelet_matrix[2:]) == 3
    homeidx, nhomeidx = np.nonzero(ishomelist)[0], np.nonzero(np.logical_not(ishomelist))[0]
    home_compv = sum(sum(reference_cube.compare(reverse_cube, flet_index=homeidx)) == 3)
    nhome_compv = sum(sum(reference_cube.compare(reverse_cube, flet_index=nhomeidx)) == 3)
    
    for i, inv_mv in reversed([(j, VectorCube.inverse(mv)) for j, mv in enumerate(moves)]):
    #{
        reverse_cube.rotate(inv_mv)
        reference_cube.rotate(inv_mv)
        for mv in VectorCube.MOVES:
        #{
            if VectorCube.inverse(mv) != inv_mv:
                pert_cube = CfopCube(reverse_cube).rotate(mv)
                hcomp = sum(sum(reference_cube.compare(pert_cube, flet_index=homeidx)) == 3)
                nhcomp = sum(sum(reference_cube.compare(pert_cube, flet_index=nhomeidx)) == 3)
                if (nhcomp > nhome_compv): perturbations.append(tuple((i, mv, CfopCube(pert_cube))))
        #}
    #}
    
    return sorted(perturbations)
#}

In [ ]:
# solve_state = FullCubeState(fsolver)
starting_cube = DirectCube().scramble()
view = CubeView(DirectCube(starting_cube))
view.push_snapshot()

cube, moves = rollout(starting_cube, fsolver, verbose=True)

view.viewable_cube.reset(state=cube.state())
view.push_snapshot(caption=f"Moves: {len(moves)} : Val: {fsolver.dance_heuristic(cube)}")
view.draw_snapshots()

In [ ]:
best_moves = []
best_value = solve_state.heuristic(starting_cube)
view = CubeView(CfopCube(starting_cube))
view.push_snapshot()

cube_list = [CfopCube(starting_cube)]
cube_list.extend([CfopCube(starting_cube).rotate(mv) for mv in VectorCube.MOVES])
cube_list.extend([CfopCube(starting_cube).rotate(mv1).rotate(mv2) for mv1 in VectorCube.MOVES for mv2 in VectorCube.MOVES])
print("Numb cubes:", len(cube_list))

start_time = time.time()
for i, rcube in enumerate(cube_list):
#{
    cube, moves = rollout(rcube, solve_state, verbose=False)
    hval = solve_state.heuristic(cube)
    if hval > best_value: 
        best_value = hval
        best_moves = moves
        view.viewable_cube.reset(state=cube.state())
        print(f"Found new best: {best_value}, from cube: {i}")
        
    if (time.time() - start_time) > 300:
        print("Progessed to iteration:", i, "in", (time.time() - start_time)/60, "min")
        start_time = time.time()
#}

view.push_snapshot(caption=f"Moves: {len(moves)} : Val: {solve_state.heuristic(view.viewable_cube)}")
view.draw_snapshots()

In [ ]:
best_cube = CfopCube(starting_cube)
view = CubeView(best_cube)
view.push_snapshot()

best_cube, moves = rollout(best_cube, solve_state)
best_value = solve_state.heuristic(best_cube)

view.viewable_cube.reset(state=best_cube.state())
view.push_snapshot(caption=f"Moves: {len(moves)} : Val: {solve_state.heuristic(best_cube)}")

for iterations in range(20):
#{
    is_new_best = False
    for i, mv, pert_cube in generate_perturbations(best_cube, moves):
    #{
        rollout_cube, moves = rollout(pert_cube, solve_state)
        rollout_value = solve_state.heuristic(rollout_cube)
        if rollout_value > best_value:
            best_cube = rollout_cube
            best_value = rollout_value
            is_new_best = True
            
            view.viewable_cube.reset(state=best_cube.state())
            view.push_snapshot(caption=f"Moves: {len(moves)} : Val: {solve_state.heuristic(rollout_cube)}")
    #}      
    
    if not is_new_best: break
#}

view.draw_snapshots()

# view = CubeView(cube).draw_projection()
# view = CubeView(best_cube).draw_projection()
# print(is_exposed, best_move, best_index)

# # Test 1
# dcube = CfopCube(starting_cube)
# view = CubeView(dcube)
# view.push_snapshot()

# for i, mv in enumerate(moves):
#     if i == best_index:
#         dcube.rotate(best_move)
#         view.push_snapshot()
#     dcube.rotate(mv)

# view.push_snapshot()
# view.draw_snapshots()

# # Test 2
# dcube = CfopCube(starting_cube)
# view = CubeView(dcube)
# view.push_snapshot()

# dcube.reset(state=best_cube.state())
# view.push_snapshot()

# dcube.rotate_seq(moves[best_index:])
# view.push_snapshot()
# view.draw_snapshots()

In [ ]:
is_exposed, best_cube, best_move, best_index = generate_perturbations(cube)
view = CubeView(cube).draw_projection()
view = CubeView(best_cube).draw_projection()
print(is_exposed, best_move, best_index)

next_cube, moves = rollout(best_cube, solve_state)
view = CubeView(next_cube).draw_projection()

# view.viewable_cube.reset(state=cube.state())
# view.push_snapshot(caption=f"Moves: {len(moves)} : Val: {solve_state.heuristic(rollout_cube)}")
# view.draw_snapshots()

In [ ]:
solve_state = FullCubeState(fsolver)
starting_cube = CfopCube().scramble()

best_value = -1
best_state = None
numb_moves = None
intro_moves = None

cube = CfopCube(starting_cube)
nmoves, bvalue, bstate = rollout(cube, solve_state)
if bvalue > best_value:
    best_value = bvalue
    best_state = bstate
    numb_moves = nmoves

print(f"From base cube: {numb_moves}, bvalue: {best_value}")
view = CubeView(cube)
view.push_snapshot()
view.viewable_cube.reset(state=best_state)
view.push_snapshot(caption=f"Moves: {numb_moves} : Val: {best_value}")
view.draw_snapshots()
    
for mv1 in CfopCube.MOVES:
#{
    cube = CfopCube(starting_cube).rotate(mv1)
    nmoves, bvalue, bstate = rollout(cube, solve_state)
    if bvalue > best_value:
        best_value = bvalue
        best_state = bstate
        numb_moves = nmoves
        intro_moves = [mv1]
        
    for mv2 in CfopCube.MOVES:
    
        if mv2[0] == mv1[0]: break
        cube = CfopCube(starting_cube).rotate(mv1).rotate(mv2)
        nmoves, bvalue, bstate = rollout(cube, solve_state)
        if bvalue > best_value:
            best_value = bvalue
            best_state = bstate
            numb_moves = nmoves
            intro_moves = [mv1, mv2]
#}

view = CubeView(CfopCube(starting_cube))
view.push_snapshot()

for mv in intro_moves:
    view.viewable_cube.rotate(mv)
    view.push_snapshot(caption=f"{color_name(mv[0])}({mv[1]})")

view.viewable_cube.reset(state=best_state)
view.push_snapshot(caption=f"Moves: {numb_moves} : Val: {best_value}")
view.draw_snapshots()

In [ ]:
direct_cube = DirectCube()
print(fsolver.order_heuristic(direct_cube))
print(fsolver.dance_heuristic(direct_cube))

direct_cube.scramble()
print(fsolver.order_heuristic(direct_cube))
print(fsolver.dance_heuristic(direct_cube))

In [ ]:
# These were in CfopSolver, but was only ever designed for a cyclic tree solve

@staticmethod
def full_solve(fsolver, csolver, cfop_cube):
#{
    UCTNode.reset_tree()
    solve_state = FullCubeState(fsolver)
    rootnode = UCTNode(solve_state.heuristic(cfop_cube))
    solved, node = CfopSolver.uc_cyclic_tree_search(csolver, cfop_cube, rootnode, solve_state)
    moves = CfopSolver.get_move_sequence(node)

    cfop_cube = CfopSolver.output_state_progress(solved, solve_state, cfop_cube, moves)
    return solved, moves, solve_state 
#}

@staticmethod
def uc_cyclic_tree_search(csolver, cfop_cube, rootnode, solve_state, max_moves=130, iterations=20):
#{
    # Catches anomoly of uc_tree_search called on already stage-solved cube
    if solve_state.is_complete(cfop_cube): return True, rootnode

    # UCT loop (w/o ROLLOUT)
    for i in range(iterations):
    #{
        winner = 0
        node = rootnode
        cube = CfopCube(cfop_cube)
        UCTNode.move_depth = 0

        # SELECT (to a frontier leaf)
        node_path_debug = [node]
        while node.child_nodes:
            node = node.select_uct_child()
            cube.rotate(node.move)
            UCTNode.move_depth += 1
            node_path_debug.append(node)

        # EXPAND (breadth and depth combo)
        if UCTNode.move_depth < max_moves:
        #{
            # Breadth first (only one layer deeper),
            # add subset of the 18 possible rotations 
            for move in solve_state.expand_moves(cube):
            #{
                if not CfopSolver.is_redundant_move(node, move):
                    child_cube = CfopCube(cube).rotate(move)
                    child_node = node.add_child(solve_state.heuristic(child_cube), move)
                    if solve_state.is_complete(child_cube): return True, child_node
            #}

            # Descend into a random initial node,
            # then cyclic-depth-search and expand
            node = random.choice(node.child_nodes)
            solved, node = CfopSolver.cycsearch(cube.rotate(node.move), solve_state, node, cyclen=30)
            if solved: return True, node
        #}

        # BACKTRACK
        while node:
            node.update()
            node = node.parent_node
    #}

    return False, rootnode.select_mvp_child()
#}

@staticmethod
def cycsearch(tree_cube, solve_state, node, cyclen=None):
#{
    cube = CfopCube(tree_cube)
    best_state = cube.state()
    best_value = UCTNode.top_value
    best_moves, all_moves = [], []

    rollout_cache = {}
    for it in range(20):
    #{
        # Explore cyclic moves
        is_new_best_state = False
        for i, moves in enumerate(csolver.cycles):
        #{
            if CfopSolver.is_redundant_move(node, moves[0]): continue

            applied_moves = []
            rollout_cube = CfopCube(cube)                
            for j in range(cyclen if cyclen else len(moves)):
            #{
                mv = moves[j%len(moves)]
                applied_moves.append(mv)
                rollout_cube.rotate(mv)

                k = tuple(rollout_cube.state())
                if k not in rollout_cache:
                    rollout_cache[k] = solve_state.heuristic(rollout_cube)                
                    if rollout_cache[k] > best_value:
                        best_value = rollout_cache[k]
                        best_moves = list(applied_moves)
                        best_state = rollout_cube.state()
                        is_new_best_state = True
            #}
        #}

        if is_new_best_state:
            cube.reset(state=best_state)
            all_moves.extend(best_moves)

        else: break
    #}

    # EXPAND
    cube = CfopCube(tree_cube)
    for cycmove in all_moves:
    #{
        for move in solve_state.expand_moves(cube):
        #{
            if (move == cycmove) and CfopSolver.is_redundant_move(node, move):
                print(f"Interesting: cycle move {move} appears to be redundant")

            if (move != cycmove) and not CfopSolver.is_redundant_move(node, move):
                child_cube = CfopCube(cube).rotate(move)
                child_node = node.add_child(solve_state.heuristic(child_cube), move)
                if solve_state.is_complete(child_cube): return True, child_node
        #}

        node = node.add_child(solve_state.heuristic(cube.rotate(cycmove)), cycmove)
        if solve_state.is_complete(cube): return True, node            
    #}

    return False, node
#}

@staticmethod
def output_running_status(node_path_list):
#{
    if (time.time() - CfopSolver.prnt_time) > 1.0:
    #{            
        output_string = ''
        for depth, node in enumerate(node_path_list):
        #{
            curr_move_str = FaceletSolver.path_str([node.move]) if node.move else 'root'
            reference_move = CfopSolver.ref_moves[depth] if depth < len(TreeSolver.ref_moves) else []

            move_options, found, index = '[', False, -1
            for idx, child in enumerate(node.get_uct_children()):
            #{
                if reference_move: found = (reference_move == child.move)
                if idx < 5: move_options += f'{FaceletSolver.path_str([child.move])}:{child.ucbonus()}/{child.value()}, '
                if found: index = idx; break
            #}

            move_options = re.sub(r', $', ']', move_options)
            ref_move_str = FaceletSolver.path_str([reference_move]) if reference_move else 'past'
            if not found: output_string += (f'[{curr_move_str}]: {ref_move_str}, NOT found in {len(node.child_nodes)} moves: {move_options}\n')
            else: output_string += (f'[{curr_move_str}]: {ref_move_str}, found at {index} of {len(node.child_nodes)}: {move_options}\n')
        #}

        clear_output(wait=True)
        print(output_string)
        TreeSolver.prnt_time = time.time()
    #}
#}

In [ ]:
view = CubeView(CfopCube(starting_cube)).push_snapshot()
for i, mv in enumerate(moves):
    if i == 12: break
    view.viewable_cube.rotate(mv)
    view.push_snapshot(caption=f"{i+1}: {color_name(mv[0])} ({mv[1]})")

view.draw_snapshots()
view.viewable_cube.state()

In [ ]:
rev_cube = CfopCube(starting_cube)
for inv in invmoves:
#{
    string = None
    max_h, min_h = 0, 1024
    
    init = fsolver.order_heuristic(rev_cube)
    for i, h_mv in enumerate(get_ordered_moves(rev_cube)):
        if max_h < h_mv[0]: max_h = h_mv[0]
        if min_h > h_mv[0]: min_h = h_mv[0]
        if inv == h_mv[1]: string = f"{i} : {h_mv} :"
            
    string += f"  min/max: ({min_h}, {max_h})"
    final = fsolver.order_heuristic(rev_cube.rotate(inv))
    string += f"  diff: {final - init} "
    print(string)
#}

print("\n")
tree_cube = CfopCube(starting_cube)
for tree_mv in moves:
#{
    string = None
    max_h, min_h = 0, 1024
    
    init = fsolver.order_heuristic(tree_cube)
    for i, h_mv in enumerate(get_ordered_moves(tree_cube)):
        if max_h < h_mv[0]: max_h = h_mv[0]
        if min_h > h_mv[0]: min_h = h_mv[0]
        if tree_mv == h_mv[1]: string = f"{i} : {h_mv} :"
            
    string += f"  min/max: ({min_h}, {max_h})"
    final = fsolver.order_heuristic(tree_cube.rotate(tree_mv))
    string += f"  diff: {final - init} "
    print(string)
#}

In [ ]:
I4 = np.identity(3, dtype=int)*4
x = np.broadcast_to(np.array([[4,0,0]]).T, (3, 72))
y = np.broadcast_to(np.array([[0,4,0]]).T, (3, 72))
z = np.broadcast_to(np.array([[0,0,4]]).T, (3, 72))

b = np.concatenate((VectorCube._ed_ring_mid, VectorCube._ed_ring_mid, VectorCube._ed_ring_mid))
a = np.concatenate((VectorCube._cn_ring_prv, VectorCube._cn_ring_nxt, np.broadcast_to(VectorCube._centers, (4, 6)).T.flatten()))

cube = CfopCube()
diff_sq = (cube.facelet_matrix[2:, a] - cube.facelet_matrix[2:, b])**2

print(sum([sum(sum((I4 == diff).T) == 3) for diff in diff_sq.T]))
print(sum(sum(np.logical_or(np.logical_or((x == diff_sq), (y == diff_sq)), (z == diff_sq))) == 3))

In [ ]:
%%time
for i in range(10000):
    cube = CfopCube().scramble(sz=20)
    diff_sq = (cube.facelet_matrix[2:, a] - cube.facelet_matrix[2:, b])**2
    sum([sum(sum((I4 == diff).T) == 3) for diff in diff_sq.T])

In [ ]:
%%time
for i in range(10000):
    cube = CfopCube().scramble(sz=20)
    diff_sq = (cube.facelet_matrix[2:, a] - cube.facelet_matrix[2:, b])**2
    sum(sum(np.logical_or(np.logical_or((x == diff_sq), (y == diff_sq)), (z == diff_sq))) == 3)

In [ ]:
cube = CfopCube()
print(cube.facelet_matrix[2:, [0,1]])
print(tuple(cube.facelet_matrix[2:, [0,1]].T.flatten()))

cube_dict = {}
for i in range(10000):
    cube = CfopCube().scramble()
    diff_sq_w = (cube.facelet_matrix[2:,0] - cube.facelet_matrix[2:,1])**2
    diff_sq_b = (cube.facelet_matrix[2:,37] - cube.facelet_matrix[2:,38])**2
    if (((sum(diff_sq_w == np.array([4, 0, 0])) == 3) or 
         (sum(diff_sq_w == np.array([0, 4, 0])) == 3) or
         (sum(diff_sq_w == np.array([0, 0, 4])) == 3)) and 
        ((sum(diff_sq_b == np.array([4, 0, 0])) == 3) or 
         (sum(diff_sq_b == np.array([0, 4, 0])) == 3) or
         (sum(diff_sq_b == np.array([0, 0, 4])) == 3))):
        cube_dict[tuple(cube.facelet_matrix[2:, [0,1]].T.flatten())] = cube.state()

# count = 1
# view = CubeView(CfopCube())
# for si, sign in itertools.product([(2,5),(1,4),(0,3)],[3, -3]):
#     for k, cstate in sorted(cube_dict.items()):
#         if (k[si[0]] == sign) and (k[si[1]] == sign):
#             view.viewable_cube.reset(state=cstate)
#             diff_sq = (view.viewable_cube.facelet_matrix[2:,0] - view.viewable_cube.facelet_matrix[2:,1])**2
#             view.push_snapshot(caption=f"{count} : {k} : {diff_sq}", flet_idx=[0,1,9,37,38, 4,13,22,31,40,49])
#             count += 1
        
# view.draw_snapshots()

###################################################################################################
# cube = CfopCube()
# view = CubeView(cube)

# for mv in [None, (WHITE_CB, -90),(WHITE_CB, -90),(WHITE_CB, -90), (WHITE_CB, -90)]:
#     if mv: cube.rotate(mv)
#     print(cube.facelet_matrix[2:, [0,1]])
#     diff_sq = (cube.facelet_matrix[2:,0] - cube.facelet_matrix[2:,1])**2
#     view.push_snapshot(caption=f"{diff_sq}", flet_idx=[0,1])

# view.draw_snapshots()

In [ ]:
def get_ordered_moves(cube):
    return sorted([(fsolver.order_heuristic(CfopCube(cube).rotate(mv)), mv) for mv in CfopCube.MOVES], reverse=True)

get_ordered_moves(CfopCube().scramble())

# Projection display
cube = CfopCube()
view = CubeView(cube)
moves, invmoves = cube.trace_scramble(sz=20)
for mv in moves: cube.rotate(mv)

for inv in invmoves:
#{
    string = None
    max_h, min_h = 0, 1024
    
    init = fsolver.order_heuristic(cube)
    for i, h_mv in enumerate(get_ordered_moves(cube)):
        if max_h < h_mv[0]: max_h = h_mv[0]
        if min_h > h_mv[0]: min_h = h_mv[0]
        if inv == h_mv[1]: string = f"{i} : {h_mv} :"
            
    string += f"  min/max: ({min_h}, {max_h})"
    final = fsolver.order_heuristic(cube.rotate(inv))
    string += f"  diff: {final - init} "
    print(string)
#}
    
# for inv in invmoves:
# #{
#     string = None
#     max_h, min_h = 0, 1024
    
#     for i, h_mv in enumerate(get_ordered_moves(cube)):
#         if max_h < h_mv[0]: max_h = h_mv[0]
#         if min_h > h_mv[0]: min_h = h_mv[0]
#         if inv == h_mv[1]: string = f"{i} {h_mv}"
            
    
#     string += f"  min/max: ({min_h}, {max_h})"
#     print(string)
#     cube.rotate(inv)
# #}

#     view.push_snapshot(caption=f"{color_name(inv[0])} ({inv[1]})")

# view.draw_snapshots()

In [ ]:
CubeView(cube).draw_projection(flet_idx=[0,1,37,38])
diff_sq_w = (cube.facelet_matrix[2:,0] - cube.facelet_matrix[2:,1])**2
diff_sq_b = (cube.facelet_matrix[2:,37] - cube.facelet_matrix[2:,38])**2

I4 = np.identity(3, dtype=int)*4
print(diff_sq_w, sum(sum((I4 == diff_sq_w).T) == 3) > 0)
print(diff_sq_b, sum(sum((I4 == diff_sq_b).T) == 3) > 0)

In [ ]:
cube = CfopCube().scramble()
diff_sq = (cube.facelet_matrix[2:, FaceletSolver._order_cn] - 
           cube.facelet_matrix[2:, FaceletSolver._order_ed])**2

print(diff_sq.shape)

one_hot_cube = np.zeros((72,60), dtype=int)
for i, diff in enumerate(diff_sq.T):
    one_hot_cube[i, NNSolver.DIFF_SQ_OH_INDEX[tuple(diff)]] = 1
    
# print(one_hot_cube)

#     for diff in diff_sq.T: 
#         k = tuple(diff)
#         diff_dict[k] = diff_dict.get(k, 0) + 1
        
#         if k not in diff_dict:
#             diff_dict[k] = one_hot_index
#             one_hot_index += 1

In [ ]:
# for k, v in sorted(diff_dict.items(), key=lambda kv: kv[1], reverse=True):
#     print(k, v)
    
# for k, v in sorted(diff_dict.items(), key=lambda kv: kv[1], reverse=True):
#     print(list(k))

for i, v in enumerate([[4,9,9],[9,4,9],[9,9,4],[4,0,0],[0,4,0],[0,0,4],[4,1,1],[1,4,1],[1,1,4],
                    [4,25,25],[25,4,25],[25,25,4],[25,4,1],[25,1,4],[4,25,1],[1,25,4],[4,1,25],[1,4,25],
                    [9,1,0],[9,0,1],[1,9,0],[0,9,1],[1,0,9],[0,1,9],[25,9,0],[25,0,9],[9,25,0],
                    [0,25,9],[9,0,25],[0,9,25],[36,4,0],[36,0,4],[4,36,0],[0,36,4],[4,0,36],[0,4,36],
                    [36,16,4],[36,4,16],[16,36,4],[4,36,16],[16,4,36],[4,16,36],[16,4,0],[16,0,4],[4,16,0],
                    [0,16,4],[4,0,16],[0,4,16],[16,9,1],[16,1,9],[9,16,1],[1,16,9],[9,1,16],[1,9,16],
                    [25,16,9],[25,9,16],[16,25,9],[9,25,16],[16,9,25],[9,16,25]]):
    print(f"{tuple(v)}: {i},")

In [ ]:
diff_dict = {}
for i in range(10000):
    cube = CfopCube().scramble()
    diff_sq_w = (cube.facelet_matrix[2:,0] - cube.facelet_matrix[2:,1])**2
    diff_sq_b = (cube.facelet_matrix[2:,37] - cube.facelet_matrix[2:,38])**2
    
    k = tuple(np.concatenate((diff_sq_w, diff_sq_b)))
    diff_dict[k] = diff_dict.get(k, 0) + 1

In [ ]:
for k, v in sorted(diff_dict.items(), key=lambda kv: kv[1], reverse=True):
    print(k, v)

In [ ]:
cstate = cube_dict[(3, 2, 2, 3, 2, 0)]
cube = CfopCube().reset(state=cstate)
CubeView(cube).draw_projection(flet_idx=[0,1,9,37,38,4,13,22,31,40,49])

# rcube = CfopCube(cube)
# rview = CubeView(rcube)
# for mv in VectorCube.MOVES:
#     rcube.rotate(mv)
#     diff_sq_w = (rcube.facelet_matrix[2:,0] - rcube.facelet_matrix[2:,1])**2
#     diff_sq_b = (rcube.facelet_matrix[2:,37] - rcube.facelet_matrix[2:,38])**2

#     if (sum(sum((I4 == diff_sq_w).T) == 3) == 0) or (sum(sum((I4 == diff_sq_b).T) == 3) == 0):
#         rview.push_snapshot(caption=f"{color_name(mv[0])}({mv[1]})", flet_idx=[0,1,9,37,38,4,13,22,31,40,49])
#         print(diff_sq_w)
#         print(diff_sq_b)
        
#     rcube.rotate(VectorCube.inverse(mv))

# rview.draw_snapshots()

rcube = CfopCube(cube)
rview = CubeView(rcube)
for mv in VectorCube.MOVES:
#{
    rcube.rotate(mv)
    diff_sq_w = (rcube.facelet_matrix[2:,0] - rcube.facelet_matrix[2:,1])**2
    diff_sq_b = (rcube.facelet_matrix[2:,37] - rcube.facelet_matrix[2:,38])**2

    if (sum(sum((I4 == diff_sq_w).T) == 3) == 0) or (sum(sum((I4 == diff_sq_b).T) == 3) == 0):
    #{
        rview.push_snapshot(caption=f"{color_name(mv[0])}({mv[1]}) {diff_sq_w} {diff_sq_b}", flet_idx=[0,1,37,38])
        print(diff_sq_w)
        print(diff_sq_b)
        
        for mv2 in VectorCube.MOVES:
        #{
            rcube.rotate(mv2)
            diff_sq_w = (rcube.facelet_matrix[2:,0] - rcube.facelet_matrix[2:,1])**2
            diff_sq_b = (rcube.facelet_matrix[2:,37] - rcube.facelet_matrix[2:,38])**2
            
            if (sum(sum((I4 == diff_sq_w).T) == 3) == 0) or (sum(sum((I4 == diff_sq_b).T) == 3) == 0):
            #{
                rview.push_snapshot(caption=f"L2: {color_name(mv[0])}({mv[1]}) {diff_sq_w} {diff_sq_b}", flet_idx=[0,1,37,38])
                print(" ", diff_sq_w)
                print(" ", diff_sq_b)
            #}
            
            rcube.rotate(VectorCube.inverse(mv2))
        #}
    #}
    
    rcube.rotate(VectorCube.inverse(mv))
#}

rview.draw_snapshots()
    
# count = 1
# view = CubeView(CfopCube())
# for si, sign in itertools.product([(2,5),(1,4),(0,3)],[3, -3]):
#     for k, cstate in sorted(cube_dict.items()):
#         if (k[si[0]] == sign) and (k[si[1]] == sign):
#             view.viewable_cube.reset(state=cstate)
#             diff_sq = (view.viewable_cube.facelet_matrix[2:,0] - view.viewable_cube.facelet_matrix[2:,1])**2
#             view.push_snapshot(caption=f"{count} : {k} : {diff_sq}", flet_idx=[0,1,9,37,38, 4,13,22,31,40,49])
#             count += 1
        
# view.draw_snapshots()

In [ ]:
cfop_cube = CfopCube(starting_cube)
CubeView(cfop_cube).draw_projection()
adapter = SMAdapter(cfop_cube)
view = CubeView(adapter)

view.record_moves()
adapter.rotate_singmaster_seq(['X2', "Y'"])
adapter.rotate_singmaster_seq(["x'", "U'", 'R', "U'", "R2'", 'F', 'x', 'R', 'U', "R'", "U'", 'R', 'B2'])

# adapter.rotate_singmaster_seq(['X2', "Y'"])
# adapter.rotate_singmaster_seq(['L2', "U'", 'L', 'B', "L'", 'U', 'L2', "U'", "r'", "U'", 'r'])

# adapter.rotate_singmaster_seq(['Y2', 'Y2'])
view.record_moves(stop_recording=True)

# Display animation
print("Rendering animation...")
HTML(view.get_animation_3d().to_jshtml())

In [ ]:
f2l_index = [i for i, flet in enumerate(CfopCube._facelet_matrix.T) 
             if (flet[0] == WHITE_CB) or ((flet[0] != YELLOW_CB) and (sum(np.equal([0,1,2,3,4,5], flet[1])) > 0))]

index = 1
for i, alg in enumerate(PLLState(fsolver).load_algorithms()):
#{
    adapter = SMAdapter(CfopCube())
    view = CubeView(adapter)
    
    f2l_solved = []
    for z_rot in ["", "Y", "Y2", "Y\'"]:
        adapter.local_cube.reset()
        adapter.rotate_singmaster('X2')
        if z_rot: adapter.rotate_singmaster(z_rot)
        invalg = [adapter.inverse(sm_name) for sm_name in reversed(alg)]
        adapter.rotate_singmaster_seq(invalg)
        f2l_solved.append(adapter.local_cube.solved(flet_index=f2l_index))
        view.push_snapshot()
    
    if sum(f2l_solved) == 0:
        print(i+1, alg, "(broken)")
        view.draw_snapshots()
#     else:
#         print(i+1, alg, "(legit)")
#         view.draw_snapshots()
#}

In [ ]:
# for i, alg in enumerate(PLLState(fsolver).load_algorithms()):
#     print(i, alg)

CubeView(cfop_cube).draw_projection()

sm_adapter = SMAdapter(CfopCube(cfop_cube))
view = CubeView(sm_adapter)
view.record_moves()

sm_adapter.rotate_singmaster_seq(['X2', 'Y2', 'Y2', 'U', 'Y2', 'Y2', 'U', 'Y2', 'Y2', 'U', 'Y2', 'Y2'])
view.record_moves(stop_recording=True)
HTML(view.get_animation_3d().to_jshtml())

In [ ]:
home = None
for i, flet in enumerate(CfopCube._facelet_matrix.T):
    if sum(flet[:2] == [WHITE_CB, 6]) == 2: home = flet[2:]; print(i, home)

pos, dots = [], []
for i, flet in enumerate(CfopCube._facelet_matrix.T):
    if sum(flet[:2] == [WHITE_CB, 6]) == 2:  dots.append(home.dot(flet[2:])); pos.append(flet[2:]); print(i, color_name(flet[0]), home.dot(flet[2:]))
    if sum(flet[:2] == [GREEN_CB, 0]) == 2:  dots.append(home.dot(flet[2:])); pos.append(flet[2:]); print(i, color_name(flet[0]), home.dot(flet[2:]))
    if sum(flet[:2] == [ORANGE_CB, 2]) == 2: dots.append(home.dot(flet[2:])); pos.append(flet[2:]); print(i, color_name(flet[0]), home.dot(flet[2:]))
    if sum(flet[:2] == [GREEN_CB, 6]) == 2:  dots.append(home.dot(flet[2:])); pos.append(flet[2:]); print(i, color_name(flet[0]), home.dot(flet[2:]))
    if sum(flet[:2] == [ORANGE_CB, 8]) == 2: dots.append(home.dot(flet[2:])); pos.append(flet[2:]); print(i, color_name(flet[0]), home.dot(flet[2:]))
    if sum(flet[:2] == [YELLOW_CB, 0]) == 2: dots.append(home.dot(flet[2:])); pos.append(flet[2:]); print(i, color_name(flet[0]), home.dot(flet[2:]))   
                                                         

In [ ]:
home = None
for i, flet in enumerate(CfopCube._facelet_matrix.T):
    if sum(flet[:2] == [WHITE_CB, 6]) == 2:  home = flet[2:]; print(f"{color_letr(flet[0])}{flet[1]} : {flet[2:]}")
    if sum(flet[:2] == [GREEN_CB, 0]) == 2:  print(f"{color_letr(flet[0])}{flet[1]} : {flet[2:]}")
    if sum(flet[:2] == [ORANGE_CB, 2]) == 2: print(f"{color_letr(flet[0])}{flet[1]} : {flet[2:]}")
    if sum(flet[:2] == [GREEN_CB, 6]) == 2:  print(f"{color_letr(flet[0])}{flet[1]} : {flet[2:]}")
    if sum(flet[:2] == [ORANGE_CB, 8]) == 2: print(f"{color_letr(flet[0])}{flet[1]} : {flet[2:]}")
    if sum(flet[:2] == [YELLOW_CB, 0]) == 2: print(f"{color_letr(flet[0])}{flet[1]} : {flet[2:]}")


print("\n")
for i, flet in enumerate(CfopCube._facelet_matrix.T):
    pos = flet[2:]
    if (home[0]*pos[0] >= 0) and (home[1]*pos[1] >= 0):
        print(f"{color_letr(flet[0])}{flet[1]} : {flet[2:]}")
    

In [ ]:
issues = {}
for i in range(10000):
    flet = CfopCube().scramble().facelet_matrix[:, 6]
    if ((sum([sum(flet[2:] == p) == 3 for p in pos]) == 0) 
        and (sum(np.equal(dots, home.dot(flet[2:]))) > 0)):
        issues[tuple(flet[2:])] = home.dot(flet[2:])

print(issues)

for i, flet in enumerate(CfopCube._facelet_matrix.T):
    if (sum(flet[2:] == (2, 3, 2)) == 3): print(i, color_name(flet[0]), ":", flet[1])
    if (sum(flet[2:] == (-3, -2, 2)) == 3): print(i, color_name(flet[0]), ":", flet[1])                                            

In [ ]:
# Use this failing cube to test F2L condition that  
# likely requires better pair management to be solved 

fail_state = np.array([-2,  3,  2, -2,  0,  3,  3, -2, -2,  2,  3,  0,  0,  0,  3, -3,  0,
       -2, -3, -2,  2,  0,  2,  3,  3, -2,  2, -2,  2,  3,  3,  2,  0, -2,
       -3,  2,  3,  0,  2,  0, -3,  0, -2,  3,  0,  2,  2,  3,  3, -2,  0,
       -2,  3, -2, -2, -2,  3,  0,  3,  2,  2, -2,  3, -3,  2,  0,  3,  0,
        0,  0,  2, -3, -3,  2, -2,  0, -2, -3, -2, -3, -2,  2, -3,  2, -2,
        0, -3,  2, -3, -2,  0,  3, -2,  0,  3,  0,  2,  0, -3, -2, -2, -3,
       -2, -3,  0,  3,  2, -2,  2, -2, -3, -3,  0,  2, -3,  2,  2,  3,  0,
       -2, -3,  0,  0,  2,  0,  3,  2,  3, -2,  0, -2,  3,  3,  2,  2, -2,
        2, -3,  0, -3, -2, -3, -2, -2,  2, -3,  0,  0,  0, -3, -3, -2,  0,
        2,  3,  2,  0, -3,  2,  2,  2, -3])

new_cube = CfopCube()
new_cube.reset(state=fail_state)
CubeView(new_cube).draw_projection()

In [ ]:
# XPAD = 200
# YPAD = 500000

# def init():
# #{
#     # Set x-axis limits in Hz over piano freq window
#     ax.set_xlim(fftbin_to_freq(fft_imin) - XPAD,
#                 fftbin_to_freq(fft_imax) + XPAD)

#     # Set y-axis limits response extrema across entire animation
#     ax.set_ylim(response_min - YPAD, response_max + YPAD)
    
#     # Zero out data buffers
#     xdata = np.zeros((fft_imax-fft_imin,))
#     ydata = np.zeros((fft_imax-fft_imin,))
#     ln.set_data(xdata, ydata)
#     return ln,
# #}

# def run_animation(frame_number):
# #{
#     ydata = spectral[frame_number][fft_imin:fft_imax]
#     xdata = np.linspace(fftbin_to_freq(fft_imin), 
#                         fftbin_to_freq(fft_imax), 
#                         fft_imax-fft_imin)
    
#     ln.set_data(xdata, ydata)
#     return ln,
# #}

# # Intialize plot and buffers
# fig, ax = plt.subplots()
# ax.set_xlabel('Hz'); ax.set_ylabel('Freq Response')
# ln, = plt.plot([], [], animated=True)
# xdata = np.zeros((fft_imax-fft_imin,))
# ydata = np.zeros((fft_imax-fft_imin,))

# # Run/render animation with approx 307.464ms time step duration
# time_step_duration = (data.shape[0]*1000)/(rate*len(spectral))
# ani = FuncAnimation(fig, run_animation, frames=np.arange(0, len(spectral)), 
#                     init_func=init, interval=time_step_duration, blit=False, repeat=False)

# # Display animation
# HTML(ani.to_jshtml())